# Mini Project: Hệ thống quản lý bán hàng với MySQL

In [15]:
import pymysql
import os
from dotenv import load_dotenv

# Load environment variables from .credential file
load_dotenv("../.credential")

# Get credentials from environment variables
conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)
cursor = conn.cursor()


In [ ]:
cursor.execute("CREATE DATABASE IF NOT EXISTS SalesManageDB;")
cursor.execute("USE SalesManageDB;")


# Tạo bảng Products
cursor.execute("""
CREATE TABLE IF NOT EXISTS PRODUCTS (
    ProductID INT PRIMARY KEY, 
    ProductName VARCHAR(100) NOT NULL,
    Category VARCHAR(50) NOT NULL,
    Price DECIMAL(10,2) NOT NULL,
    StockQuantity INT NOT NULL          
);
""")

# Tạo bảng CUSTOMERS
cursor.execute("""
CREATE TABLE IF NOT EXISTS CUSTOMERS (
    CustomerID INT PRIMARY KEY,
    FullName VARCHAR(100) NOT NULL,
    PhoneNumber VARCHAR(15) UNIQUE NOT NULL,        
    Email VARCHAR(100) UNIQUE NOT NULL
);
""")

# Tạo bảng SALESPERSONS
cursor.execute("""
CREATE TABLE IF NOT EXISTS SALESPERSONS (
    SalespersonID INT PRIMARY KEY,
    FullName VARCHAR(100) NOT NULL,
    PhoneNumber VARCHAR(15) UNIQUE NOT NULL        
);
""")

# Tạo bảng ORDERS
cursor.execute("""
CREATE TABLE IF NOT EXISTS ORDERS (
    OrderID INT PRIMARY KEY,
    CustomerID INT NOT NULL,
    SalespersonID INT NOT NULL,
    OrderDate DATE NOT NULL,
    TotalAmount DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (CustomerID) REFERENCES CUSTOMERS(CustomerID) ON DELETE CASCADE,
    FOREIGN KEY (SalespersonID) REFERENCES SALESPERSONS(SalespersonID) ON DELETE CASCADE

);
""")

# Tạo bảng ORDERDETAILS
cursor.execute("""
CREATE TABLE IF NOT EXISTS ORDERDETAILS (
    OrderDetailID INT PRIMARY KEY,
    OrderID INT NOT NULL,
    ProductID INT NOT NULL,
    Quantity INT NOT NULL,
    Subtotal DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (OrderID) REFERENCES ORDERS(OrderID) ON DELETE CASCADE,
    FOREIGN KEY (ProductID) REFERENCES PRODUCTS(ProductID) ON DELETE CASCADE

);
""")

# Chèn dữ liệu vào bảng PRODUCTS
cursor.execute("""
INSERT INTO PRODUCTS (ProductID, ProductName, Category, Price, StockQuantity) VALUES
(1, 'Laptop Dell XPS', 'Laptop', 25000000, 10),
(2, 'iPhone 14', 'Điện thoại', 22000000, 15),
(3, 'Tai nghe Sony', 'Phụ kiện', 3000000, 20),
(4, 'Bàn phím Cd', 'Phụ kiện', 1500000, 50),
(5, 'MacBook Air M2', 'Laptop', 28000000, 8);
""")

# Chèn dữ liệu vào bảng CUSTOMERS
cursor.execute("""
INSERT INTO CUSTOMERS (CustomerID, FullName, PhoneNumber, Email) VALUES
(1, 'Nguyễn Văn A', '0987654321', 'a@example.com'),
(2, 'Trần Thị B', '0976543210', 'b@example.com'),
(3, 'Lê Văn C', '0965432109', 'c@example.com'),
(4, 'Phạm Thị D', '0945678123', 'd@example.com'),
(5, 'Bùi Văn E', '0936789123', 'e@example.com');
""")

# Chèn dữ liệu vào bảng SALESPERSONS
cursor.execute("""
INSERT INTO SALESPERSONS (SalespersonID, FullName, PhoneNumber) VALUES
(1, 'Lê Hoàng Nam', '0934567890'),
(2, 'Trần Quốc Bảo', '0945678123'),
(3, 'Nguyễn Minh Hà', '0956789123');
""")

# Chèn dữ liệu vào bảng ORDERS
cursor.execute("""
INSERT INTO ORDERS (OrderID, CustomerID, SalespersonID, OrderDate, TotalAmount) VALUES
(1, 1, 1, '2024-03-10', 25000000),
(2, 2, 2, '2024-03-11', 3000000),
(3, 3, 2, '2024-03-12', 22000000);
""")

# Chèn dữ liệu vào bảng ORDERDETAILS
cursor.execute("""
INSERT INTO ORDERDETAILS (OrderDetailID, OrderID, ProductID, Quantity, Subtotal) VALUES
(1, 1, 1, 1, 25000000),
(2, 2, 3, 1, 3000000),
(3, 3, 2, 1, 22000000);
""")

# Commit thay đổi vào cơ sở dữ liệu
conn.commit()


In [17]:
# Truy vấn danh sách tất cả sản phẩm còn hàng
cursor.execute("""
SELECT ProductName, StockQuantity FROM PRODUCTS WHERE StockQuantity > 0;
""")

data = cursor.fetchall()
print(data)

(('Laptop Dell XPS', 10), ('iPhone 14', 15), ('Tai nghe Sony', 20), ('Bàn phím Cd', 50), ('MacBook Air M2', 8))


In [18]:
# Truy vấn danh sách đơn hàng của khách hàng có CustomerID = 1
cursor.execute("""
SELECT * from orders WHERE 1 = 1 and customerid = 1;
""")

data = cursor.fetchall()
print(data)

((1, 1, 1, datetime.date(2024, 3, 10), Decimal('25000000.00')),)


In [19]:
# Truy vấn tổng doanh số của từng nhân viên
cursor.execute("""
SELECT salespersons.fullname, salespersons.salespersonid, sum(orders.totalamount) 
               as sales_person_revenue from salespersons 
               inner join orders on salespersons.salespersonid = orders.salespersonid group by salespersonid;
""")

data = cursor.fetchall()
print(data)

(('Lê Hoàng Nam', 1, Decimal('25000000.00')), ('Trần Quốc Bảo', 2, Decimal('25000000.00')))


In [21]:
# Nhân viên bán hàng có tổng doanh số cao nhất
cursor.execute("""
SELECT salespersons.fullname, salespersons.salespersonid, sum(orders.totalamount) 
               as sales_person_revenue from salespersons 
               inner join orders on salespersons.salespersonid = orders.salespersonid group by salespersonid 
               order by sales_person_revenue desc limit 1;
""")

data = cursor.fetchall()
print(data)

(('Lê Hoàng Nam', 1, Decimal('25000000.00')),)


In [22]:
# Tổng doanh thu của từng mục sản phẩm
cursor.execute("""
select products.productid, products.productname, orderdetails.quantity*orderdetails.subtotal 
               as product_revenue from products 
               inner join orderdetails 
               on products.productid = orderdetails.productid;
""")

data = cursor.fetchall()
print(data)

((1, 'Laptop Dell XPS', Decimal('25000000.00')), (3, 'Tai nghe Sony', Decimal('3000000.00')), (2, 'iPhone 14', Decimal('22000000.00')))


In [23]:
# Top 3 khách hàng có tổng chi tiêu cao nhất
cursor.execute("""
select customers.customerid, customers.fullname, orders.totalamount 
               as customer_spend 
               from customers 
               inner join orders on customers.customerid = orders.customerid 
               order by customer_spend desc 
               limit 3;
""")

data = cursor.fetchall()
print(data)

((1, 'Nguyễn Văn A', Decimal('25000000.00')), (3, 'Lê Văn C', Decimal('22000000.00')), (2, 'Trần Thị B', Decimal('3000000.00')))


In [24]:
# Tìm tháng có doanh thu cao nhất
cursor.execute("""
SELECT orderdate, totalamount as month_revenue from orders order by month_revenue desc limit 1;
""")

data = cursor.fetchall()
print(data)

((datetime.date(2024, 3, 10), Decimal('25000000.00')),)


In [25]:
# Tìm sản phẩm bán chạy nhất theo số lượng
cursor.execute("""
select products.productid, products.productname, orderdetails.quantity 
               from products 
               inner join orderdetails on products.productid = orderdetails.productid 
               order by quantity desc 
               limit 1;
""")

data = cursor.fetchall()
print(data)

((1, 'Laptop Dell XPS', 1),)


In [26]:
# Close Connection
cursor.close()
conn.close()